# ASTR 19 Final Project

Tutorial that was followed for the project: [Link](https://sep.readthedocs.io/en/v1.0.x/tutorial.html)

Documentation for the `astropy.io.fits` package: [Link](https://docs.astropy.org/en/stable/io/fits/)

Import packages needed for the project

In [ ]:
import numpy as np
from astropy.io import fits
import sep
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Ellipse

Set `rcParams` for the format of the plots when using `matplotlib`

In [ ]:
rcParams["figure.figsize"] = [10., 8.]

Since we are using `astropy.io.fits`, we first get the HDU List from the `image.fits` file that we are using. The `image.fits` file is from the sep GitHub account ([Link here](https://github.com/kbarbary/sep/blob/main/data/image.fits)). HDU stands for 'Header Data Units' and it is the "fundamental container structure of the FITS format consisting of a `data` member and its associated metadata in a `header`" ([reference](https://docs.astropy.org/en/stable/io/fits/api/hdus.html)).

This cell opens the file and puts it into an HDU List object. The `info()` method was used to check the contents of the HDU List.

In [ ]:
hdul = fits.open("image.fits")
hdul.info()

Here we get the data we are working with from the HDU List and put it into the `data` variable.

In [ ]:
data = hdul[0].data

The `matplotlib` library was used to graph the data from the `image.fits` file. First, `numpy` was used to calculate the mean and the standard deviation of the data. Then, those values were used to set the parameters for the plot. A `colorbar` was also created on the side of the graph.

In [ ]:
mean, std = np.mean(data), np.std(data)
plt.imshow(data, interpolation="nearest", cmap="gray", vmin=(mean - std), vmax=(mean + std), origin="lower")
plt.colorbar()

# save the figure as a png
plt.savefig("tut_01.png", bbox_inches='tight', dpi=400)

The `Background()` function gets a "representation of spatially variable image background and noise" ([reference](https://sep.readthedocs.io/en/v1.0.x/api/sep.Background.html)).

The `bg` stores a `Background` object.

In [ ]:
bg = sep.Background(data)

The `back()` function does the equivalent of representing the `Background` object as a 2-d `numpy` array.

In [ ]:
bg_img = bg.back()

Plot the background with `matplotlib`. Add a `colorbar` on the side.

In [ ]:
plt.imshow(bg_img, interpolation="nearest", cmap="gray", origin="lower")
plt.colorbar()

# save the figure as a png
plt.savefig("tut_02.png", bbox_inches='tight', dpi=400)

Create an array of the background rms and store it in the `bg_rms` variable.

In [ ]:
bg_rms = bg.rms()

Plot the background noise with `matplotlib`. Add a `colorbar` on the side.

In [ ]:
plt.imshow(bg_rms, interpolation="nearest", cmap="gray", origin="lower")
plt.colorbar()

# save the figure as a png
plt.savefig("tut_03.png", bbox_inches='tight', dpi=400)

Subtract the background from the data. Doing `data - bg` instead of `bg.subfrom(data)` to avoid modifying values in-place (this preserves the original data).

In [ ]:
data_sub = data - bg

Run object detection using the `sep.extract()` function on the background-subtracted data.

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bg.globalrms)

Count the number of objects detected.

In [ ]:
print(len(objects))

In [ ]:
# background-subtracted image
fig, ax = plt.subplots()
mean, std = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation="nearest", cmap="gray", vmin=(mean - std), vmax=(mean + std), origin="lower")

# plot an ellipse for each object found
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=(6 * objects['a'][i]), height=(6 * objects['b'][i]), angle=(objects['theta'][i] * 180. / np.pi))
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

# save the figure as a png
plt.savefig("tut_04.png", bbox_inches='tight', dpi=400)

In [ ]:
objects.dtype.names

In [ ]:
flux, flux_err, _ = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bg.globalrms, gain=1.0)

In [ ]:
for i in range(10):
    print(f"object {i}: flux = {flux[i]} +/- {flux_err[i]}")

Close the HDU List object when done using it.

In [ ]:
hdul.close()